In [195]:
from requests_html import HTMLSession
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from time import sleep
import re
from ast import literal_eval

In [4]:
vol = range(1,15)

In [ ]:
url_dict = {}
for v in vol:
    print(v, f"https://www.parlimen.gov.my/arkib-ahli.html?uweb=dr&arkib=yes&vol={v}")
    with HTMLSession() as session:
        r= session.get(f"https://www.parlimen.gov.my/arkib-ahli.html?uweb=dr&arkib=yes&vol={v}", timeout=5)
        html = r.html
        url_dict[v] = html.xpath("//*[@id = 'contentBoxText']/ul/li/div/a/@href")


In [ ]:
raw_maklumat = pd.DataFrame()

In [20]:
for vol_, url_list in tqdm(url_dict[14].items()):
# vol_ = 14.0
# url_list = url_dict[14][166:]
    for url in tqdm(url_list, leave=False):
        # print(vol_, url)
        try:
            maklumat_ahli_df = pd.read_html(f"https://www.parlimen.gov.my/{url}")[1].iloc[3:].T
        except UnicodeDecodeError:
            maklumat_ahli_df = pd.read_html(f"https://www.parlimen.gov.my/{url}", encoding='cp1252')[1].iloc[3:].T
            maklumat_ahli_df.loc[1,3] = bytes(maklumat_ahli_df.loc[1,3].replace("’", "'"),"cp1252").decode("utf-8")
        except:
            sleep(10)
            maklumat_ahli_df = pd.read_html(f"https://www.parlimen.gov.my/{url}")[1].iloc[3:].T

        maklumat_ahli_df = maklumat_ahli_df.rename(columns=maklumat_ahli_df.iloc[0]).drop(maklumat_ahli_df.index[0]).reset_index(drop=True)

        if vol_ >= 12:
            with HTMLSession() as session:
                r = session.get(f"https://www.parlimen.gov.my/{url}", timeout=5)
                html = r.html
                ahli_info = html.xpath('//table//table')
                maklumat_ahli_df.loc[0, "Gambar"] = [ahli_info[0].xpath("//img/@src")[0]]
        else:
            maklumat_ahli_df.loc[0, "Gambar"] = None
        maklumat_ahli_df.loc[0, "vol"] = vol_

        raw_maklumat = pd.concat([raw_maklumat,maklumat_ahli_df])
        raw_maklumat = raw_maklumat.reset_index(drop=True)
        raw_maklumat.to_csv("raw_maklumat.csv", index=False)
        sleep(1)

  0%|          | 0/56 [00:00<?, ?it/s]

In [121]:
raw_maklumat.columns=['Nama','Jawatan_dalam_Kabinet','Tempat_Duduk','Parlimen','Kawasan',
'Negeri','No_Telefon','No_Faks','Email','Alamat_Surat-menyurat',
'Gambar','vol','Parti','Jawatan_dalam_Parlimen','Hubungi']
raw_maklumat.to_csv("raw_maklumat.csv", index=False)

In [500]:
def jantina(full_name):
    gender_l = set(["HAJI", "TUAN", "HJ.", "BIN", "A/L"])
    gender_p = set(["HAJAH","HAJJAH", "HJH","PUAN", "DATIN", "BINTI", "A/P"])
    
    male = False
    female = False
    prep_name = full_name.upper().split(" ")
    for c in ["BIN", "BINTI", "@"]:
        if c in prep_name:
            sep_loc = np.where([True if i == c else False for i in prep_name])[0].max()
            prep_name = prep_name[:sep_loc+1]
    prep_name = set(prep_name)
    if prep_name.intersection(gender_l):
        male = True
    if prep_name.intersection(gender_p):
        female = True
    if male and female:
        if full_name in ["Tuan Hajah Ilani Binti Dato' Haji Isahak", "Tuan Shariffah Dorah binti Mohammed", "Tuan Wan Zainab Binti M. A. Bakar"]:
            male = False
        else:
            print(prep_name)
            print(f"male and female identifiers detected for {full_name}")
    
    if male and not female:
        return "male"
    elif female and not male:
        return "female"
    else:
        return None

def etnik(fullname):
    bumi = False
    india = False
    others = False
    bumi_identifier = set(["BIN", "BINTI", "ANAK", "ABDUL","SYED"])
    india_identifier = set(["A/L", "A/P"])
    others_indentifier = set(["SINGH"])
    prep_name = fullname.upper().split(" ")
    prep_name = set(prep_name)
    if prep_name.intersection(bumi_identifier):
        bumi = True
    if prep_name.intersection(india_identifier):
        india = True
    if prep_name.intersection(others_indentifier):
        others = True

    if bumi and not india and not others:
        return "bumiputera"
    elif india and not bumi and not others:
        return "india"
    elif others and not bumi and not india:
        return "others"
    else:
        return None
    

def name_gelaran_splitting(vol, fullname):
    
    name_fix = {"TuanAbdul": "Tuan Abdul", "TuanYong":"Tuan Yong", "Dato'Jamaluddin":"Dato' Jamaluddin"}
    for pattern,subbed in name_fix.items():
        fullname = re.sub(pattern, subbed, fullname)
    # manual_fix = {"Y.A.B. Dato Musa Hitam,":"Y.A.B. Dato Musa Hitam","Datuk Leonard Linggi Anak Jugah,":"Datuk Leonard Linggi Anak Jugah"}
    # if fullname in manual_fix.keys()
    # print(fullname)
    separator_agama = ["HAJI", "HJ.", "HAJAH","HAJJAH", "HJH"]
    separator_common = ["TUAN", "PUAN", "CIK"]
    separator_prof = ["TS.", "TS", "IR.", "IR", "DR.", "DR", "DRS", "DRS." "(DR)", "(DR.)", "PROF.", "PROF"]
    separator_tentera = ["(B)", "KOMANDER", "JEN", "BRIG", "LT.","LT", "KOL", "KOL.", "KAPTEN"]
    separator_timur = ["PENGHULU"]
    separator_diraja = ["TENGKU", "TUA", "CHEGU", "PENGARAH"]
    separator_besar = [ "NEGARA", "DIRAJA", "UTAMA", "PANGLIMA", "WIRA", "SERI", "SRI", "TEMENGGUNG", "TEMENGGONG", "SETIA", "PAHLAWAN", "INDERA", "PADUKA", "PATINGGI", "AMAR", 'DATIN', 'DATO', "DATO'", 'DATO’', 'DATU', 'DATUK', 'TUN', 'TUNKU',"Y.A.B."]
    all_separator = separator_agama + separator_common + separator_prof + separator_tentera + separator_timur + separator_diraja + separator_besar

    if vol in [12.0, 13.0] and re.findall(",", fullname):
        return fullname.split(",")[-1].strip(), fullname.split(",")[0].strip()
        
    else:
        name_split_upper = fullname.upper().split(" ")  
        name_split = fullname.split(" ")  

        for c in ["BIN", "BINTI", "@", "ANAK"]:
            if c in name_split_upper:
                sep_loc = np.where([True if i == c else False for i in name_split_upper])[0].max()
                name_split_upper = name_split_upper[:sep_loc]
        # print(np.where([True if i in all_separator else False for i in name_split_upper]))
        gelaran_sep_index = np.where([True if i in all_separator else False for i in name_split_upper])[0]
        
        if gelaran_sep_index.size > 0:
            gelaran_sep = gelaran_sep_index.max()
            # print(f"gel: {gelaran_sep}")
            gelaran =  name_split[gelaran_sep] if gelaran_sep == 0 else " ".join(name_split[:gelaran_sep+1])
            nama_cleaned = " ".join(fullname.split(" ")[gelaran_sep + 1:])
        else:
            gelaran = None
            nama_cleaned = fullname
        return gelaran, nama_cleaned

def clean_names(raw_maklumat):
    def clean_dr(tx):
        list_split = tx.group(0).split("Dr.")
        return "Dr. ".join(list_split)

    for i in raw_maklumat[raw_maklumat.Nama.str.contains(r'[Dd][Rr]\.[^\s,")]')].index:
        # print(raw_maklumat.loc[i, "Nama"])
        to_replace = raw_maklumat.loc[i, "Nama"]
        cleaned_name = re.sub(r'[Dd][Rr]\.[^\s,")]', clean_dr, to_replace)
        raw_maklumat.loc[i, "Nama"] = cleaned_name

    for i in raw_maklumat[raw_maklumat.Nama.str.contains(r',$')].index:
        # print(raw_maklumat.loc[i, "Nama"])
        to_replace = raw_maklumat.loc[i, "Nama"]
        cleaned_name = re.sub(r',$', "", to_replace)
        raw_maklumat.loc[i, "Nama"] = cleaned_name
    
    return raw_maklumat



In [487]:
raw_maklumat = pd.read_csv("raw_maklumat.csv")
raw_maklumat.loc[:, "Gambar"] = raw_maklumat.Gambar.str.replace("['","").str.replace("']","")
raw_maklumat = clean_names(raw_maklumat)
all_ahli = []
for maklumat_ahli in raw_maklumat.itertuples():
    gelaran, nama_cleaned = name_gelaran_splitting(maklumat_ahli.vol, maklumat_ahli.Nama)
    ahli_row = {}
    ahli_row["vol"] = maklumat_ahli.vol
    ahli_row["kod_kawasan"] = maklumat_ahli.Parlimen
    ahli_row["negeri"] = maklumat_ahli.Negeri
    ahli_row["kawasan"] = maklumat_ahli.Kawasan
    ahli_row["gelaran"] = gelaran
    ahli_row["nama"] = nama_cleaned
    ahli_row["tahun_lahir"] = None
    ahli_row["jantina"] = jantina(maklumat_ahli.Nama)
    ahli_row["kumpulan_etnik"] = etnik(maklumat_ahli.Nama)
    ahli_row["parti"] = maklumat_ahli.Parti
    ahli_row["jawatan_kabinet"] = maklumat_ahli.Jawatan_dalam_Kabinet
    ahli_row["jawatan_parlimen"] = maklumat_ahli.Jawatan_dalam_Parlimen
    ahli_row["gambar"] = f"https://www.parlimen.gov.my{maklumat_ahli.Gambar}" if not pd.isna(maklumat_ahli.Gambar) else None
    all_ahli.append(ahli_row)

In [488]:
pd.DataFrame.from_records(all_ahli).to_parquet("cleaned_maklumat.parquet")

In [489]:
pd.DataFrame.from_records(all_ahli).to_csv("cleaned_maklumat.csv", index=False)
pd.DataFrame.from_records(all_ahli).to_csv("DR.csv", index=False)

# CURRENT

In [297]:
with HTMLSession() as session:
    r= session.get(f"https://www.parlimen.gov.my/ahli-dewan.html?uweb=dr&", timeout=5)
    html = r.html
    list_v15 = html.xpath("//*[@class='card mb-0']/*/ul/li/div/a/@href")


In [308]:
raw_maklumat_new = pd.DataFrame()

In [309]:
vol_ = 15.0
for url in tqdm(list_v15, leave=False):
    # print(vol_, url)
    try:
        maklumat_ahli_df = pd.read_html(f"https://www.parlimen.gov.my/{url}")[1].iloc[3:].T
    except UnicodeDecodeError:
        maklumat_ahli_df = pd.read_html(f"https://www.parlimen.gov.my/{url}", encoding='cp1252')[1].iloc[3:].T
        maklumat_ahli_df.loc[1,3] = bytes(maklumat_ahli_df.loc[1,3].replace("’", "'"),"cp1252").decode("utf-8")
    except:
        sleep(10)
        maklumat_ahli_df = pd.read_html(f"https://www.parlimen.gov.my/{url}")[1].iloc[3:].T

    maklumat_ahli_df = maklumat_ahli_df.rename(columns=maklumat_ahli_df.iloc[0]).drop(maklumat_ahli_df.index[0]).reset_index(drop=True)

    if vol_ >= 12:
        with HTMLSession() as session:
            r = session.get(f"https://www.parlimen.gov.my/{url}", timeout=5)
            html = r.html
            ahli_info = html.xpath('//table//table')
            maklumat_ahli_df.loc[0, "Gambar"] = [ahli_info[0].xpath("//img/@src")[0]]
    else:
        maklumat_ahli_df.loc[0, "Gambar"] = None
    maklumat_ahli_df.loc[0, "vol"] = vol_

    raw_maklumat_new = pd.concat([raw_maklumat_new,maklumat_ahli_df])
    raw_maklumat_new = raw_maklumat_new.reset_index(drop=True)
    raw_maklumat_new.to_csv("raw_maklumat_new.csv", index=False)
    sleep(1)

  0%|          | 0/224 [00:00<?, ?it/s]

In [315]:
raw_maklumat_new.columns=['Nama','Jawatan_dalam_Kabinet','Tempat_Duduk','Parlimen','Kawasan',
'Negeri','No_Telefon','No_Faks','Email','Alamat_Surat-menyurat',
'Gambar','vol','Parti','Jawatan_dalam_Parlimen','Hubungi']
raw_maklumat_new.to_csv("raw_maklumat_new.csv", index=False)

In [493]:
raw_maklumat_new = pd.read_csv("raw_maklumat_new.csv")
raw_maklumat_new.loc[:, "Gambar"] = raw_maklumat_new.Gambar.str.replace("['","").str.replace("']","")
raw_maklumat_new = clean_names(raw_maklumat_new)
all_ahli_new = []
for maklumat_ahli in raw_maklumat_new.itertuples():
    gelaran, nama_cleaned = name_gelaran_splitting(maklumat_ahli.vol, maklumat_ahli.Nama)
    ahli_row = {}
    ahli_row["vol"] = maklumat_ahli.vol
    ahli_row["kod_kawasan"] = maklumat_ahli.Parlimen
    ahli_row["negeri"] = maklumat_ahli.Negeri
    ahli_row["kawasan"] = maklumat_ahli.Kawasan
    ahli_row["gelaran"] = gelaran
    ahli_row["nama"] = nama_cleaned
    ahli_row["tahun_lahir"] = None
    ahli_row["jantina"] = jantina(maklumat_ahli.Nama)
    ahli_row["kumpulan_etnik"] = etnik(maklumat_ahli.Nama)
    ahli_row["parti"] = maklumat_ahli.Parti
    ahli_row["jawatan_kabinet"] = maklumat_ahli.Hubungi
    ahli_row["jawatan_parlimen"] = maklumat_ahli.Jawatan_dalam_Kabinet
    ahli_row["gambar"] = f"https://www.parlimen.gov.my{maklumat_ahli.Gambar}" if not pd.isna(maklumat_ahli.Gambar) else None
    all_ahli_new.append(ahli_row)

In [494]:
pd.DataFrame.from_records(all_ahli_new).to_csv("cleaned_maklumat_new.csv", index=False)

In [495]:
pd.DataFrame.from_records(all_ahli_new).to_parquet("cleaned_maklumat_new.parquet")
pd.DataFrame.from_records(all_ahli_new).to_csv("DR_15.csv", index=False)

In [331]:
"TEST"[0:1]

'T'

# <11

In [502]:
dr_old = pd.read_csv("DR - DR 1-11.csv")

In [508]:
dr_old.loc[:,"kumpulan_etnik"] = dr_old.loc[:,"nama"].apply(etnik)

In [509]:
dr_old

,vol,kod_kawasan,negeri,kawasan,gelaran,nama,tahun_lahir,jantina,kumpulan_etnik,parti,jawatan_kabinet,jawatan_parlimen,gambar
0,1,NaN,NaN,Kuala Kedah,Y.T.M. Tunku,Abdul Rahman Putra AlHaj,NaN,male,bumiputera,NaN,Perdana Menteri,NaN,NaN
1,1,NaN,NaN,Pekan,Y.A.B. Tun,Abdul Razak Bin Dato' Hussain,NaN,male,bumiputera,NaN,Timbalan Perdana Menteri,NaN,NaN
2,1,NaN,NaN,Johor Timur,Dato' Dr.,Ismail Bin Dato' Abdul Rahman,NaN,male,bumiputera,NaN,NaN,NaN,NaN
3,1,NaN,NaN,Sungei Siput,Dato',V.T. Sambanthan,NaN,male,None,NaN,NaN,NaN,NaN
4,1,NaN,NaN,Muar Selatan,Dato',Suleiman Bin Dato' Abdul Rahman,NaN,male,bumiputera,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1841,11,NaN,NaN,Pendang,Dr.,Mohamed Hayati Bin Othman,NaN,male,bumiputera,NaN,NaN,NaN,NaN
1842,11,NaN,NaN,Batu Gajah,Puan,Fong Po Kuan,NaN,female,None,NaN,NaN,NaN,NaN
1843,11,NaN,NaN,Bagan,Tuan,Lim Hock Seng,NaN,male,None,NaN,NaN,NaN,NaN
1844,11,NaN,NaN,Bandar Kuching,Tuan,Chong Chieng Jen,NaN,male,None,NaN,NaN,NaN,NaN


In [525]:
dr_new = pd.read_csv("DR - DR 12-12 with ID.csv")

In [526]:
kawasan_mapping = dr_new[["kod_kawasan","negeri","kawasan"]].dropna()

In [527]:
kawasan_mapping["kawasan"] = kawasan_mapping["kawasan"].str.upper()

In [528]:
kawasan_mapping = kawasan_mapping.drop_duplicates().reset_index(drop=True)

In [529]:
kawasan_mapping

,kod_kawasan,negeri,kawasan
0,P004,Kedah,LANGKAWI
1,P064,Perak,IPOH TIMUR
2,P162,Johor,GELANG PATAH
3,P162,Johor,ISKANDAR PUTERI
4,P051,Pulau Pinang,BUKIT GELUGOR
...,...,...,...
247,P117,Kuala Lumpur,SEGAMBUT
248,P118,Kuala Lumpur,SETIAWANGSA
249,P119,Kuala Lumpur,TITIWANGSA
250,P121,Kuala Lumpur,LEMBAH PANTAI


In [ ]:
dr_old["kawasan_to_map"]